In [1]:
import numpy as np
import pandas as pd

In [2]:
variables = np.array(
    [
        "a","b","c","d","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u",
        "v","w","x","y","z"
    ]
)
digits = np.array(["0","1","2","3","4","5","6","7","8","9"])
arithmetic = np.array(["-","+","*","/"])
equalities = ["==", ">", "<", ">=", "<="]
power = "**"
assignment = "="
if_start = "if CONDITION:\n"
if_else = "else:\n"
if_elif = "elif CONDITION:\n"
list_comprehension = "[FUNCTION(VARIABLE) for VARIABLE in LIST]"
function_start = "def NAME(ARGUMENTS):\n"
function_return = "return(NAME)"

Expression: variable = many digits or arithmetic or power or variable

In [3]:
def make_digit():
    s = "".join(
        np.random.choice(digits, size=np.random.choice(np.arange(1,5)))
    )
    if s[0] == "0" and len(s) > 1:
        s = s[1:]
    return(s)

In [4]:
def init_variable(current_variables):
    if np.random.choice([True, False]):
        s = np.random.choice(variables)+" = "+random_expression()
    else:
        if len(current_variables) > 0:
            s = np.random.choice(current_variables)+" = "+random_expression()
        else:
            s = np.random.choice(variables)+" = "+random_expression()
    return(s)

In [5]:
def random_expression():
    s = make_digit()
    while np.random.choice([True, False]):
        if np.random.choice([True, False]):
            s += np.random.choice(arithmetic)
            s += make_digit()
    return(s)

In [6]:
def make_condition(current_variables):
    if len(current_variables) > 0:
        var = np.random.choice(current_variables)
    else:
        var = np.random.choice(variables)
    s = var + " " + np.random.choice(equalities) + " "
    what = np.random.choice(["variable", "digit", "expression"])
    if what == "variable":
        if len(current_variables):
            s += np.random.choice(current_variables)
        else:
            s += np.random.choice(variables)
    elif what == "digit":
        s += make_digit()
    else:
        s += random_expression()
    return(s)

In [7]:
n = np.random.choice(np.arange(5,10))

In [27]:
def make_code(n):
    code = []
    current_variables = []
    open_ifs = []
    block_count = 0
    prev = "start"
    for a in range(n):
        what = np.random.choice(
            [
                "assignment", "if_start", "if_elif", "if_else", "b++", "b--"
            ], p = [0.5, 0.3, 0.09, 0.09, 0.01, 0.01]
        )
        if what == "assignment":
            line = init_variable(current_variables)
            current_variables.append(line[0])
            current_variables = list(set(current_variables))
            code.append(block_count*"    ")
            code.append(line)
        elif what == "b++":
            block_count += 1
        elif what == "b++":
            block_count -= 1
        elif what == "if_start":
            if a != n-1 and n != 0:
                code.append(block_count*"    ")
                code.append(if_start.replace("CONDITION", make_condition(current_variables)))
                block_count += 1
                open_ifs.append(block_count)
            else:
                code.append(block_count*"    ")
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        elif what == "if_elif":
            if prev == "assignment" and a != n-1 and n != 0:
                if len(open_ifs) > 0:
                    if open_ifs[-1] == block_count:
                        code.append(
                            if_elif.replace("CONDITION", make_condition(current_variables))
                        )
                        block_count += 1
            else:
                if len(open_ifs) > 0:
                    block_count = open_ifs[-1]
                code.append(block_count*"    ")
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        elif what == "if_else":
            if prev == "assignment" and a != n-1 and n != 0:
                if len(open_ifs) > 0:
                    if open_ifs[-1] == block_count:
                        code.append(
                            if_else.replace("CONDITION", make_condition(current_variables))
                        )
                        block_count += 1
            else:
                if len(open_ifs) > 0:
                    block_count = open_ifs[-1]
                code.append(block_count*"    ")
                line = init_variable(current_variables)
                current_variables.append(line[0])
                current_variables = list(set(current_variables))
                code.append(line)
        code += "\n"
        prev = what
      #  print(a, what)
    if len(current_variables) > 0:
        to_output = np.random.choice(current_variables)
    else:
        to_output = np.random.choice(variables)
    code += "print("+np.random.choice(current_variables)+")"
    return(code, current_variables)

In [9]:
import re
import os
import subprocess
import os.path as op
from tqdm import tqdm

In [10]:
if not op.exists("tasks"):
    os.makedirs("tasks")

In [11]:
if not op.exists("pdfs"):
    os.makedirs("pdf")

In [12]:
if not op.exists("tex"):
    os.makedirs("tex")

In [13]:
if not op.exists("answers"):
    os.makedirs("answers")

In [14]:
if not op.exists("variables"):
    os.makedirs("variables")

In [23]:
N_tasks = 400

In [28]:
for a in range(N_tasks):
    fn = op.join("tasks", str(a)+".py")
    ans_fn = op.join("answers", str(a)+".txt")
    var_fn = op.join("variables", str(a)+".txt")
    code, vs = make_code(np.random.choice(np.arange(5,20)))
    with open(fn, "w") as oh:
        oh.write("".join(code))
    the_code = subprocess.Popen(
        "python "+fn, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True
    )
    stdout, stderr = the_code.communicate()
    with open(ans_fn, "w") as oh:
        if len(stdout) != 0:
            print(stdout)
            oh.write("correct\n")
            oh.write(stdout.decode())
        else:
            oh.write("error\n")
            oh.write(stderr.decode())
    with open(var_fn, "w") as oh:
        oh.write(",".join(vs))

b'64\n'
b'49\n'
b'864\n'
b'3\n'
b'62\n'
b'752\n'
b'1\n'
b'6\n'
b'8\n'
b'2830\n'
b'-98\n'
b'155\n'
b'1\n'
b'99\n'
b'390\n'
b'4715\n'
b'4324.277511961723\n'
b'8804\n'
b'0.07655978125776783\n'
b'6851\n'
b'3648\n'
b'21330.000075251948\n'
b'3169\n'
b'7441\n'
b'90\n'
b'562\n'
b'47\n'
b'63\n'
b'7952\n'
b'59\n'
b'8171\n'
b'8075\n'
b'2850\n'
b'1\n'
b'47.666666666666664\n'
b'0.0071633237822349575\n'
b'98\n'
b'8\n'
b'12\n'
b'62\n'
b'67\n'
b'4273\n'
b'1320\n'
b'78\n'
b'36.903225806451616\n'
b'4206\n'
b'625\n'
b'931\n'
b'8923\n'
b'8918\n'
b'593\n'
b'771610\n'
b'69\n'
b'772\n'
b'83\n'
b'1502\n'
b'412.18032786885243\n'
b'651\n'
b'3655\n'
b'9\n'
b'79\n'
b'4332\n'
b'9\n'
b'699\n'
b'9476\n'
b'28.000210304833097\n'
b'11\n'
b'1\n'
b'0.043243243243243246\n'
b'3\n'
b'7\n'
b'4830\n'
b'9363\n'
b'917\n'
b'647\n'
b'9\n'
b'7973\n'
b'4611\n'
b'459\n'
b'73\n'
b'6051\n'
b'3176\n'
b'25\n'
b'0\n'
b'587\n'
b'360\n'
b'668\n'
b'367\n'
b'6956\n'
b'5474\n'
b'373\n'
b'9\n'
b'3581\n'
b'704\n'
b'6478\n'
b'628\n'
b'432\n'
b'-

In [29]:
def form_answers(n):
    ans_fn = op.join("answers", str(n)+".txt")
    with open(ans_fn, "r") as ih:
        last_line = ih.readlines()[-1]
    return(last_line)

In [38]:
def form_incorrect_answers(n, corrects, chosen):
    var_fn = op.join("variables", str(n)+".txt")
    with open(var_fn, "r") as ih1:
        #chosen = np.random.choice(corrects)
        out = chosen
        variables_in_file = ih1.read().replace("\n", "").split(",")
        what = np.random.choice(["chosen", "random"])
        if what == "chosen":
            if "name" in out:
                if len(variables_in_file)>1:
                    rpl = np.random.choice(variables_in_file)
                    out = re.sub("name \'[a-z]\'", "name \'"+rpl+"\'", chosen)
        elif what == "random":
            out = make_digit()
    return(chosen)

In [39]:
form_answers(6)

'SyntaxError: invalid syntax\n'

In [44]:
form_incorrect_answers(6, correct_answers, np.random.choice(correct_answers))

'2850\n'

In [33]:
correct_answers = [form_answers(a) for a in np.arange(N_tasks)]

In [48]:
possible_incorrect_answers = list(np.unique(correct_answers))

In [ ]:
"NameError: name 'a' is not defined\n",

In [51]:
possible_incorrect_answers = list(filter(lambda x: "NameError" not in x, possible_incorrect_answers))

In [36]:
len(correct_answers)

400

In [57]:
all_answers = []
for i,a in tqdm(enumerate(correct_answers)):
    current_answers = [a]
    three_choices = np.random.choice(possible_incorrect_answers, size=3, replace=False)
    incorrect1 = form_incorrect_answers(i, correct_answers, three_choices[0])
    incorrect2 = form_incorrect_answers(i, correct_answers, three_choices[1])
    incorrect3 = form_incorrect_answers(i, correct_answers, three_choices[2])
    current_answers.extend(
        [incorrect1, incorrect2, incorrect3]
    )
    all_answers.append(np.random.permutation(current_answers))

400it [00:00, 3365.73it/s]


In [58]:
all_answers

[array(['3169\n', '64\n', '59\n', '49\n'], dtype='<U5'),
 array(['543\n', '155\n', "NameError: name 'u' is not defined\n", '64\n'],
       dtype='<U35'),
 array(['-690\n', '3427\n', '587\n', 'SyntaxError: invalid token\n'],
       dtype='<U27'),
 array(['910\n', '49\n', '4611\n', '1254\n'], dtype='<U5'),
 array(['63\n', '8075\n', 'IndentationError: unexpected indent\n',
        '2921\n'], dtype='<U36'),
 array(['480\n', '864\n', '910\n', '4206\n'], dtype='<U5'),
 array(['12\n', '0.07655978125776783\n', 'SyntaxError: invalid syntax\n',
        'SyntaxError: invalid token\n'], dtype='<U28'),
 array(['0.04098391250793177\n', "NameError: name 'a' is not defined\n",
        '75\n', '0.043243243243243246\n'], dtype='<U35'),
 array(['3427\n', "NameError: name 'k' is not defined\n", '63\n', '7\n'],
       dtype='<U35'),
 array(['3655\n', '8918\n', '3\n', '75\n'], dtype='<U5'),
 array(['130\n', '69\n', '699\n', 'IndentationError: unexpected indent\n'],
       dtype='<U36'),
 array(['9\n', "Name

In [60]:
with open("test_form.tex", "r") as ih:
    form = ih.read()

In [61]:
variant = np.random.choice(np.arange(N_tasks), 10)

In [62]:
variant

array([ 29, 342, 298,  88, 116, 213, 169, 382, 171, 327])

In [63]:
from copy import deepcopy

In [64]:
def get_variant():
    variant = np.random.choice(np.arange(N_tasks), 10)
    current_form = deepcopy(form)
    for i,a in enumerate(variant):
        anssss = all_answers[a]
        question_number = "Q"+str(i)
        current_form = current_form.replace(question_number, str(a))
        answer_number_base = "A"+str(i)
        for j,b in enumerate(anssss):
            current_form = current_form.replace(answer_number_base+str(j+1), b)
        filename = "test.py}%"+str(i)
        current_form = current_form.replace(filename, op.join("../tasks", str(a)+".py}"))
    return(current_form)

In [65]:
for a in tqdm(range(30)):
    with open(op.join("tex/", str(a)+".tex"), "w") as oh:
        cf = get_variant()
        oh.write(cf.replace("VARIANT", str(a)))
        os.system(
            "pdflatex -synctex=1 -interaction=nonstopmode "+str(a)+".tex"
        )

100%|██████████| 30/30 [00:02<00:00, 10.75it/s]


In [69]:
mergepdf = "pdfunite "
for a in tqdm(range(30)):
    mergepdf += str(a)+".pdf "

100%|██████████| 30/30 [00:00<00:00, 209715.20it/s]


In [70]:
mergepdf+"merged_output.pdf"

'pdfunite 0.pdf 1.pdf 2.pdf 3.pdf 4.pdf 5.pdf 6.pdf 7.pdf 8.pdf 9.pdf 10.pdf 11.pdf 12.pdf 13.pdf 14.pdf 15.pdf 16.pdf 17.pdf 18.pdf 19.pdf 20.pdf 21.pdf 22.pdf 23.pdf 24.pdf 25.pdf 26.pdf 27.pdf 28.pdf 29.pdf merged_output.pdf'

In [66]:
for a in tqdm(range(30)):
    print("pdflatex -synctex=1 -interaction=nonstopmode "+str(a)+".tex")

100%|██████████| 30/30 [00:00<00:00, 38468.09it/s]

pdflatex -synctex=1 -interaction=nonstopmode 0.tex
pdflatex -synctex=1 -interaction=nonstopmode 1.tex
pdflatex -synctex=1 -interaction=nonstopmode 2.tex
pdflatex -synctex=1 -interaction=nonstopmode 3.tex
pdflatex -synctex=1 -interaction=nonstopmode 4.tex
pdflatex -synctex=1 -interaction=nonstopmode 5.tex
pdflatex -synctex=1 -interaction=nonstopmode 6.tex
pdflatex -synctex=1 -interaction=nonstopmode 7.tex
pdflatex -synctex=1 -interaction=nonstopmode 8.tex
pdflatex -synctex=1 -interaction=nonstopmode 9.tex
pdflatex -synctex=1 -interaction=nonstopmode 10.tex
pdflatex -synctex=1 -interaction=nonstopmode 11.tex
pdflatex -synctex=1 -interaction=nonstopmode 12.tex
pdflatex -synctex=1 -interaction=nonstopmode 13.tex
pdflatex -synctex=1 -interaction=nonstopmode 14.tex
pdflatex -synctex=1 -interaction=nonstopmode 15.tex
pdflatex -synctex=1 -interaction=nonstopmode 16.tex
pdflatex -synctex=1 -interaction=nonstopmode 17.tex
pdflatex -synctex=1 -interaction=nonstopmode 18.tex
pdflatex -synctex=1 -i

In [146]:
with open("test_form_result.tex", "w") as oh:
    oh.write(current_form)

In [131]:
anssss

array(['7917224\n', 'SyntaxError: invalid token\n', '7', '769\n'],
      dtype='<U27')

In [115]:
correct_answers[32]

"NameError: name 'q' is not defined\n"

In [116]:
incorrect_answers_1[32]

'7359'

In [117]:
incorrect_answers_2[32]

'72'

In [118]:
incorrect_answers_3[32]

"NameError: name 'g' is not defined\n"